# LangChain 문서 분할 샘플 코드 설명

---

## 1. 코드의 목적

- **LangChain 문서로더(TextLoader)와 텍스트 분할기(CharacterTextSplitter)를 활용해**
  - 텍스트 파일을 불러오고,
  - 지정된 크기(chunk size)와 겹침(chunk overlap) 기준으로
  - 문서를 여러 개의 "청크"로 분할하는 실전 예시를 보여줍니다.
- RAG(검색 증강 생성) 및 대규모 문서 검색에 필수적인 "문서 분할"의 기본 원리를 이해하게 하는 것이 목적입니다.

---

## 2. 실무 활용 포인트
  - 문서 분할은 RAG, 벡터 DB 저장, 검색 성능 개선에 핵심적인 역할
  - 다양한 언어/문서/길이에도 맞춤형 파라미터 설정으로 유연하게 적용 가능

In [10]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

# 텍스트 파일 생성 (예제용)
sample_text = """
LangChain은 언어 모델을 활용한 애플리케이션 개발을 위한 프레임워크입니다.
다양한 구성 요소들을 체인으로 연결하여 복잡한 작업을 수행할 수 있습니다.
프롬프트 템플릿, 메모리, 에이전트 등의 기능을 제공합니다.
LCEL(LangChain Expression Language)을 통해 더 직관적인 체인 구성이 가능합니다.
벡터 저장소와 임베딩을 통해 RAG 시스템을 쉽게 구축할 수 있습니다.
"""

with open("sample.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)

# CharacterTextSplitter 주요 옵션 상세 설명

---

## 1. chunk_size

- **정의:**  
  한 번에 분할되는 "청크(chunk)"의 최대 문자 수를 지정합니다.
- **목적:**  
  - LLM 또는 벡터DB가 한 번에 처리할 수 있는 최대 길이에 맞춰  
    문서를 작은 단위(청크)로 나누기 위함입니다.
- **예시:**  
  `chunk_size=100`이면, 각 청크는 **최대 100자**까지 포함합니다.

---

## 2. chunk_overlap

- **정의:**  
  분할 시 앞뒤 청크가 **겹치는 부분의 문자 수**를 지정합니다.
- **목적:**  
  - 의미 단절 최소화:  
    청크 경계에 중요한 정보가 끊기는 문제를 완화합니다.
  - 맥락 유지:  
    이전 청크와 겹치는 정보를 포함해 LLM의 응답 품질을 높입니다.
- **예시:**  
  `chunk_overlap=20`이면,  
  이전 청크의 **마지막 20자**가 다음 청크의 **처음 20자**와 중복됩니다.
  즉 이전 청크의 **마지막 20자와 다음 청크의 처음 20자가 동일한 내용**으로 한 번만 중복되어 들어가는 구조

---

## 3. separator

- **정의:**  
  청크를 나눌 때 **우선적으로 기준으로 삼는 구분 문자(혹은 문자열)**를 지정합니다.
- **목적:**  
  - 가급적 의미 단위(문단, 문장, 줄바꿈 등)에서 청크가 끊기게 하여  
    자연스러운 분할을 지원합니다.
- **예시:**  
  `separator="\n"`이면,  
  줄바꿈(\n)을 기준으로 우선 분할하며, 만약 `chunk_size`를 초과하면 그 안에서 추가로 분할됩니다.

---

## 4. 실무 적용 팁

- **chunk_size**는 LLM 토큰 한계, 벡터DB 검색 성능 등을 고려해 설정 (일반적으로 256~1024 문자)
- **chunk_overlap**은 너무 작으면 맥락이 끊기고, 너무 크면 중복 저장/비용 증가 (20~100 문자 권장)
- **separator**는 언어, 문서 구조(줄바꿈, 문단, 문장 등)에 따라 조정

---

## 5. 한 줄 요약

> `CharacterTextSplitter`의 `chunk_size`, `chunk_overlap`, `separator` 옵션은  
> "문서의 의미와 맥락을 최대한 보존하면서  
> 벡터DB/LLM이 효율적으로 처리할 수 있도록  
> 최적의 단위로 텍스트를 분할"하기 위해 사용하는 핵심 파라미터입니다.


In [11]:



# 문서 로드
loader = TextLoader("sample.txt", encoding="utf-8")
documents = loader.load()

print("=== 원본 문서 ===")
print(f"문서 수: {len(documents)}")
print(f"문서 내용: {documents[0].page_content[:100]}...")

# 텍스트 분할
text_splitter = CharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    separator="\n"
)
docs = text_splitter.split_documents(documents)


=== 원본 문서 ===
문서 수: 1
문서 내용: 
LangChain은 언어 모델을 활용한 애플리케이션 개발을 위한 프레임워크입니다.
다양한 구성 요소들을 체인으로 연결하여 복잡한 작업을 수행할 수 있습니다.
프롬프트 템플릿, 메...


In [ ]:
print(f"\n=== 분할된 문서 ({len(docs)}개 청크) ===")
for i, doc in enumerate(docs):
    print(f"=== 청크 {i+1} ===")
    print(doc.page_content)
    print(f"메타데이터: {doc.metadata}")
    print()

# 간단한 텍스트 분할 예제 (문서 로더 없이)
simple_text = "LangChain은 AI 개발 프레임워크입니다. 다양한 기능을 제공합니다. 체인, 에이전트, RAG 등을 지원합니다."

simple_splitter = CharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=5,
    separator=". "
)

simple_chunks = simple_splitter.split_text(simple_text)
print("=== 간단한 텍스트 분할 ===")
for i, chunk in enumerate(simple_chunks):
    print(f"청크 {i+1}: {chunk}")

Created a chunk of size 25, which is longer than the specified 10
Created a chunk of size 13, which is longer than the specified 10



=== 분할된 문서 (3개 청크) ===
=== 청크 1 ===
LangChain은 언어 모델을 활용한 애플리케이션 개발을 위한 프레임워크입니다.
다양한 구성 요소들을 체인으로 연결하여 복잡한 작업을 수행할 수 있습니다.
메타데이터: {'source': 'sample.txt'}

=== 청크 2 ===
프롬프트 템플릿, 메모리, 에이전트 등의 기능을 제공합니다.
LCEL(LangChain Expression Language)을 통해 더 직관적인 체인 구성이 가능합니다.
메타데이터: {'source': 'sample.txt'}

=== 청크 3 ===
벡터 저장소와 임베딩을 통해 RAG 시스템을 쉽게 구축할 수 있습니다.
메타데이터: {'source': 'sample.txt'}

=== 간단한 텍스트 분할 ===
청크 1: LangChain은 AI 개발 프레임워크입니다
청크 2: 다양한 기능을 제공합니다
청크 3: 체인, 에이전트, RAG 등을 지원합니다.
